In [39]:
#%run /home/jovyan/data_loading_tools.ipynb
%run /home/jovyan/obs_reader.ipynb
%run /home/jovyan/plotting_tools.ipynb

In [40]:
# Client täytyy käynnistää ekana
client = start_client()

In [41]:
# Alustetaan datan hakua
producer = 'flash'
time_range = 1

delayed_data_ec = get_obs_data_delay(producer, time_range)
print(delayed_data_ec)
flash_data = merge_obs_delayed(client, delayed_data_ec)

Delayed('merge_delayed-3767e5c1-0810-4c4c-a6dd-77f91a320700')
Done!


In [42]:
flash_data.head().compute()

<xarray.Dataset>
Dimensions:     (index: 5)
Coordinates:
  * index       (index) int64 0 1 2 3 4
Data variables:
    station_id  (index) int64 35015903 35015904 35015905 35015906 35015907
    longitude   (index) float64 0.8859 1.117 1.92 1.889 0.9449
    latitude    (index) float64 59.69 58.95 58.92 58.0 61.13
    utctime     (index) object '20240829T005539' ... '20240829T015551'
    elevation   (index) object None None None None None
    flash       (index) int64 -8 -13 -12 -8 6

In [ ]:
# # Otetaan data ulos xarrayna, koska vain yksi käyttäjä DASK-järjestelmässä ja serverin käyttö hankaloittaa tekemistä porukalla
T2m = flash_data['air_temperature_4'].compute()
lat = flash_data['lat'].compute()
lon = flash_data['lon'].compute()
time = flash_data['time'].compute()

type: getitem-fcedbd4502b24779fe8df4a57607c0e7

In [ ]:
# Satunnaista testailua serveriltä
print(flash_data['air_temperature_4'].max(dim="time").values.compute())
print(flash_data['air_temperature_4'].groupby("time.hour").mean().values.compute())
print(flash_data['air_temperature_4'].resample(time="1D").compute())


[[296.36    297.61    299.36    ... 306.72    306.75    306.88   ]
 [296.36    296.97998 298.55    ... 306.59    306.61    306.75   ]
 [297.11    298.18    299.03998 ... 306.58    306.58    306.71   ]
 ...
 [281.47998 281.53    281.6     ... 281.75    281.75    281.75   ]
 [281.50998 281.53    281.59    ... 281.65    281.65    281.75   ]
 [281.50998 281.55    281.61    ... 281.65    281.65    281.65   ]]
[[[289.75998 289.54272 289.13367 ... 291.91907 292.00635 291.91092]
  [289.73453 289.64996 289.29816 ... 291.77274 291.85    291.7682 ]
  [289.48816 289.27274 289.0509  ... 291.6718  291.65454 291.6718 ]
  ...
  [279.52637 279.5618  279.61453 ... 279.59274 279.52365 279.51724]
  [279.41272 279.46725 279.5091  ... 279.58636 279.51453 279.5109 ]
  [279.3409  279.37546 279.40906 ... 279.61725 279.53452 279.53452]]

 [[289.63663 289.41    289.0379  ... 291.98166 292.06833 292.00452]
  [289.63864 289.53818 289.20407 ... 291.84998 291.93182 291.8988 ]
  [289.39548 289.21106 288.95105 ... 291

In [ ]:
# Satunnaista testailua lokaalilla

T_max = (T2m.max(dim="time").compute())
#print(T_max.values)
T_hour = (T2m.groupby("time.hour").mean().compute())
#print(T_hour.values)
T_days = (T2m.resample(time="1D"))
#print(T_days)

#print(T2m.isel(time=5).values)


# Meteorologit voi aloittaa tekemisen tästä kohtaa. Ylimpänä oleva "solu" pitää ajaa, jotta saadaan metodit käyttöön

In [ ]:
# Lasketaan tähän keskiarvoja.
# Kelluva keskiarvo

# Koko jakson yli minimi
# Koko jakson yli maksimi

# Aika-askeleet 5 vuorokauden ajalle


# Tästä eteenpäin tehdään visualisointeja

In [ ]:
# Plotataan suoraan serveriltä
def draw(parameter, t):    
    # Luetaan ja käsitellään dataa
    chart = chart = flash_data[parameter].isel(time=t).compute()
    if parameter == 'air_temperature_4':
        chart = chart - 273.15
        cmap = 'nipy_spectral'
        vmin = -40
        vmax = 40

    # Luodaan karttapohja ja kuva
    axx = get_map_layout()

    # Plotataan itse data
    plot_map_xarray(chart, vmin, vmax, cmap, axx)

In [ ]:
# Luodaan widget
widgets.interact(draw, parameter=['air_temperature_4'], t=(0, len(flash_data['time'].compute()) - 1))
plt.show()

interactive(children=(Dropdown(description='parameter', options=('air_temperature_4',), value='air_temperature…